In [1]:
import torch

In [129]:
import torch
from collections import Counter
import random
import re

file_path = '/Users/jaafarbendriss/First-LLM/dataset/black_hole_wiki.txt'

# Open the file and read its content
with open(file_path, 'r') as file:
    file_content = file.read()

raw_text = file_content

def add_start_end_sentence(text):
    return "start-of-sentence " + text.replace("."," end-of-sentence start-of-sentence")


def remove_citations(text):
    pattern = r"\[\d+\]"
    cleaned_text = re.sub(pattern, "", text)    
    return cleaned_text

def remove_spaces(text):
    pattern = r"\s+"
    cleaned_text = re.sub(pattern, " ", text)    
    return cleaned_text

def preprocess(text: str):
    return remove_spaces(remove_citations(add_start_end_sentence(text).lower().replace(" s ","").replace("\\","").replace("'"," ").replace('"',' ').replace(","," ").replace("\ s"," ").replace("\n"," ")))

preprocessed_text = preprocess(raw_text)

Counter(preprocessed_text.split(" ")).most_common(20)

preprocessed_text_array = preprocessed_text.split(" ")


print(preprocessed_text_array[0:100])

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

tokens = tokenizer(preprocessed_text)

preprocessed_text = preprocessed_text.replace("end-of-sentence start-of-sentence", "end-of-sentence69696969start-of-sentence")
splited_data = preprocessed_text.split("69696969")

train_data =  [(i,sentence) for i, sentence in zip( range(0,len(splited_data)), splited_data  )]
train_iter = iter(train_data)


['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return', 'boundary', 'for', 'other', 'uses', 'see', 'black', 'hole', '(disambiguation)', 'end-of-sentence', 'start-of-sentence', 'direct', 'radio', 'image', 'of', 'a', 'supermassive', 'black', 'hole', 'at', 'the', 'core', 'of', 'messier', '87', 'animated', 'simulation', 'of', 'a', 'schwarzschild', 'black', 'hole', 'with', 'a', 'galaxy', 'passing', 'behind', 'end-of-sentence', 'start-of-sentence', 'around', 'the', 'time', 'of', 'alignment', 'extreme', 'gravitational', 'lensing', 'of', 'the', 'galaxy', 'is', 'observed', 'end-of-sentence', 'start-of-sentence', 'a', 'black', 'hole', 'is', 'a', 'region', 'of', 'spacetime', 'where', 'gravity', 'is', 'so', 'strong', 'that', 'nothing', 'including', 'light', 'and', 'other', 'electromagnetic', 'waves', 'has', 'enough', 'energy', 'to', 'escape', 'it', 'end-of-sentence', 'start-of-sentence', 'einstein', 's', 'theory', 'of', 'general', 'relativity', 'predicts', 'that', 'a', 'sufficiently', 'co

In [130]:
next(train_iter)

(0,
 'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence')

## Preparing training dataset using sliding window

In [131]:
sentence =  'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence'

In [132]:
data = sentence.split()

In [133]:
splited_data

['start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence',
 'start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild black hole with a galaxy passing behind end-of-sentence',
 'start-of-sentence around the time of alignment extreme gravitational lensing of the galaxy is observed end-of-sentence',
 'start-of-sentence a black hole is a region of spacetime where gravity is so strong that nothing including light and other electromagnetic waves has enough energy to escape it end-of-sentence',
 'start-of-sentence einstein s theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole end-of-sentence',
 'start-of-sentence the boundary of no escape is called the event horizon end-of-sentence',
 'start-of-sentence a black hole has a great effect on the fate and circumstances of an object crossing it but it h

In [134]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
vocab(tokens[0:10])

[4, 37, 10, 66, 6, 0, 186, 16, 68, 0]

In [135]:
token_map = vocab.get_stoi()

In [136]:
splited_data[20]

'start-of-sentence the presence of a black hole can be inferred through its interaction with other matter and with electromagnetic radiation such as visible light end-of-sentence'

In [137]:
training_data = []

for sentence in splited_data:
    data=sentence.split()
    for i in range(1,len(data)):
        X=data[0:i]
        y = data[i]
        #print(i, X)
        #print(y)
        #print()
        training_data.append((X,y))

In [138]:
len(training_data)

11538

In [139]:
training_data[110:115]

[(['start-of-sentence', 'the', 'boundary', 'of', 'no', 'escape', 'is'],
  'called'),
 (['start-of-sentence',
   'the',
   'boundary',
   'of',
   'no',
   'escape',
   'is',
   'called'],
  'the'),
 (['start-of-sentence',
   'the',
   'boundary',
   'of',
   'no',
   'escape',
   'is',
   'called',
   'the'],
  'event'),
 (['start-of-sentence',
   'the',
   'boundary',
   'of',
   'no',
   'escape',
   'is',
   'called',
   'the',
   'event'],
  'horizon'),
 (['start-of-sentence',
   'the',
   'boundary',
   'of',
   'no',
   'escape',
   'is',
   'called',
   'the',
   'event',
   'horizon'],
  'end-of-sentence')]

In [140]:
word_to_token = token_map

def word_to_token_wrapper(word: str) -> int:
    if word not in word_to_token:
        return word_to_token["<unk>"]
    return word_to_token[word]

def sentence_to_token(sentence)->int:
    return [word_to_token_wrapper(word) for word in sentence.split()]

In [141]:
sentence_to_token("black")

[5]

In [142]:
word_to_token["<unk>"]

0

In [143]:
training_data

[(['start-of-sentence'], 'object'),
 (['start-of-sentence', 'object'], 'that'),
 (['start-of-sentence', 'object', 'that'], 'has'),
 (['start-of-sentence', 'object', 'that', 'has'], 'a'),
 (['start-of-sentence', 'object', 'that', 'has', 'a'], 'no-return'),
 (['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return'],
  'boundary'),
 (['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return', 'boundary'],
  'for'),
 (['start-of-sentence',
   'object',
   'that',
   'has',
   'a',
   'no-return',
   'boundary',
   'for'],
  'other'),
 (['start-of-sentence',
   'object',
   'that',
   'has',
   'a',
   'no-return',
   'boundary',
   'for',
   'other'],
  'uses'),
 (['start-of-sentence',
   'object',
   'that',
   'has',
   'a',
   'no-return',
   'boundary',
   'for',
   'other',
   'uses'],
  'see'),
 (['start-of-sentence',
   'object',
   'that',
   'has',
   'a',
   'no-return',
   'boundary',
   'for',
   'other',
   'uses',
   'see'],
  'black'),
 (['start-of-sentence',
  

In [144]:
training_data_tokenized = [ (sentence_to_token(a),word_to_token_wrapper(b) ) for a,b in training_data]

AttributeError: 'list' object has no attribute 'split'

In [145]:
training_data_tokenized[110:115]

[([4, 1, 186, 2, 92, 138, 12], 115),
 ([4, 1, 186, 2, 92, 138, 12, 115], 1),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1], 26),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26], 29),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29], 3)]

training_data_tokenized[110:115]
"""
[([4, 6], 8),
 ([4, 6, 5], 66),
 ([4, 6, 5, 8], 6),
 ([4, 6, 5, 8, 66], 412),
 ([4, 6, 5, 8, 66, 6], 153)]
 """

## training 

In [146]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

class SentenceCompletionDataset(Dataset):
    def __init__(self, data, word_to_index):
        self.data = data
        self.word_to_index = word_to_index
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_sequence, target_word = self.data[idx]
        input_indices = [self.word_to_index[word] for word in input_sequence]
        target_index = self.word_to_index[target_word]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_index, dtype=torch.long)



In [147]:
from sklearn.model_selection import train_test_split

# Assuming train_data is a list of sentences
train_sentences, val_sentences = train_test_split(masked_sentences_and_targets_tokens, test_size=0.2, random_state=42)

# Now, apply the masking and preparation logic to both train_sentences and val_sentences

NameError: name 'masked_sentences_and_targets_tokens' is not defined

In [148]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn



In [149]:
from torch.utils.data import Dataset, DataLoader

class TokenDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        tokens, target = self.data[idx]
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(target, dtype=torch.long)

# Assuming `data` is your list of mappings
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [150]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    tokens, targets = zip(*batch)
    # Pad the sequences to the maximum length in the batch
    tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)
    targets = torch.tensor(targets, dtype=torch.long)
    return tokens_padded, targets


In [151]:
len(training_data_tokenized)

11538

In [152]:
training_data_tokenized[0:5]

[([4], 37),
 ([4, 37], 10),
 ([4, 37, 10], 66),
 ([4, 37, 10, 66], 6),
 ([4, 37, 10, 66, 6], 0)]

In [89]:
vocab_size

2145

In [153]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, tokens):
        embedded = self.embedding(tokens)  # [batch_size, seq_len, embedding_dim]
        # Aggregate embeddings, e.g., by averaging
        aggregated = embedded.mean(dim=1)  # [batch_size, embedding_dim]
        hidden = self.relu(self.linear1(aggregated))  # [batch_size, hidden_dim]
        output = self.linear2(hidden)  # [batch_size, output_dim]
        return output



In [154]:
# Hyperparameters
vocab_size = 2145  # Adjust based on your dataset
embedding_dim = 50
hidden_dim = 100
output_dim = vocab_size  # Same as vocab size for prediction

model = SimpleLM(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [155]:
len(training_data_tokenized)

11538

In [156]:
import random

data = random.choices(training_data_tokenized,k=5000)
#training_data_tokenized

In [106]:
len(data)

5000

In [225]:
data = training_data_tokenized[0:3000]

In [226]:
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=15, shuffle=True, collate_fn=collate_fn)


In [227]:
num_epochs = 350  # Or whatever suits your dataset

for epoch in range(num_epochs):
    total_loss = 0
    for tokens, target in dataloader:
        optimizer.zero_grad()
        output = model(tokens)
        loss = criterion(output, target.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    if epoch %10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')


/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_70416/763346120.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)


Epoch 1, Loss: 4.497990543246269
Epoch 11, Loss: 2.4339363777637484
Epoch 21, Loss: 2.030162851884961
Epoch 31, Loss: 1.732696186825633
Epoch 41, Loss: 1.4818178392201662
Epoch 51, Loss: 1.2782090443372727
Epoch 61, Loss: 1.0624578026682139
Epoch 71, Loss: 0.9203124928846955
Epoch 81, Loss: 0.7775048652291298
Epoch 91, Loss: 0.6998862168472261
Epoch 101, Loss: 0.5782810395210981
Epoch 111, Loss: 0.5178025118110235
Epoch 121, Loss: 0.4669135155994445
Epoch 131, Loss: 0.41618850306840616
Epoch 141, Loss: 0.3880883290350903
Epoch 151, Loss: 0.36100849462411133
Epoch 161, Loss: 0.3467500051733805
Epoch 171, Loss: 0.3209612768984516
Epoch 181, Loss: 0.3036342859386059
Epoch 191, Loss: 0.2919390382153506
Epoch 201, Loss: 0.28059554990000835
Epoch 211, Loss: 0.27506257002514756
Epoch 221, Loss: 0.26045753646794767
Epoch 231, Loss: 0.2568761546628957
Epoch 241, Loss: 0.24755402850560132
Epoch 251, Loss: 0.24109351339773638
Epoch 261, Loss: 0.2438068606458546
Epoch 271, Loss: 0.2349076308117719

In [228]:
[([4, 1, 186, 2, 92, 138, 12], 115),
 ([4, 1, 186, 2, 92, 138, 12, 115], 1),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1], 26),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26], 29),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29], 3)]

[([4, 1, 186, 2, 92, 138, 12], 115),
 ([4, 1, 186, 2, 92, 138, 12, 115], 1),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1], 26),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26], 29),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29], 3)]

In [229]:
token_to_word = {token: word for word, token in word_to_token.items()}

def token_to_sentence(tokens):
    return " ".join([token_to_word[t] for t in tokens])

In [230]:
token_to_sentence([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29])

'start-of-sentence the boundary of no escape is called the event horizon'

In [231]:
sentence_to_token("black")

[5]

In [232]:
sequence = [4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29]
sequence_tensor = torch.tensor(sequence).unsqueeze(0) 
sequence_tensor

tensor([[  4,   1, 186,   2,  92, 138,  12, 115,   1,  26,  29]])

In [233]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    output = model(sequence_tensor)
    predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
predicted_token_id

3

In [245]:
output.argmax(dim=1)

tensor([3])

In [234]:
token_to_word[3]

'end-of-sentence'

In [252]:
def predict_missing_word(sentence="black hole is the biggest object in the"):
    sequence = sentence_to_token(sentence)
    # Example sequence
    #sequence = [37, 10, 66, 8, 0, 3]  # assuming this sequence with the masked token
    sequence_tensor = torch.tensor(sequence).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        output = model(sequence_tensor)
        predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
        _, top5_indices = torch.topk(output[0], 5)
        predicted_token_id = random.choice(top5_indices.tolist())

    # Assuming you have a token_to_word dictionary
    token_to_word = {token: word for word, token in word_to_token.items()}
    predicted_word = token_to_word[predicted_token_id]
    return predicted_word

In [253]:
predict_missing_word()

'that'

In [254]:
next_word = "start-of-sentence"
words = []
for _ in range(20):
    words.append(next_word)
    next_word = predict_missing_word(next_word)

" ".join(words)

'start-of-sentence general black holes <unk> the first black hole by his 1960s that limit the hole by of by of'